In [1]:
import os 
import PIL
import numpy as np

In [2]:
image_data=[]               #CREATING A LIST TO STORE THE IMAGE PIXEL VALUES.
label=[]                    #CREATING THE LIST TO STORE THE LABELS OF EACH IMAGE.

# **ITERATING OVER THE DATASET**
for dirname, _, filenames in os.walk('/kaggle/input/spider-ml-inductions-2023-task-1-part-1/trainingSet'):
    a=str(dirname)
    for filename in filenames:
        if str(filename).endswith("a.py") == False:
            img=PIL.Image.open(os.path.join(dirname, filename))
            if img is not None:
                image_data.append(np.array(img))
                label.append(a[-1])
            
image_data=np.array(image_data)/255
label=np.array(label)
final_output=[[1,0,0,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0,0,0],[0,0,1,0,0,0,0,0,0,0],[0,0,0,1,0,0,0,0,0,0],[0,0,0,0,1,0,0,0,0,0],[0,0,0,0,0,1,0,0,0,0],[0,0,0,0,0,0,1,0,0,0],[0,0,0,0,0,0,0,1,0,0],[0,0,0,0,0,0,0,0,1,0],[0,0,0,0,0,0,0,0,0,1]]
one_hot_encoded_vector=[]
for i in label:
    
    index=int(i)
    one_hot_encoded_vector.append(final_output[index])
one_hot_encoded_vector=np.array(one_hot_encoded_vector)    

LOADING THE TEST DATASET

In [3]:
image_data_1=[]               #CREATING A LIST TO STORE THE IMAGE PIXEL VALUES.
label_1=[]                    #CREATING THE LIST TO STORE THE LABELS OF EACH IMAGE.

# **ITERATING OVER THE DATASET**
for dirname, _, filenames in os.walk('/kaggle/input/spider-ml-inductions-2023-task-1-part-1/test_set'):
    for filename in filenames:
        if str(filename).endswith("a.py") == False:
            img=PIL.Image.open(os.path.join(dirname, filename))
            if img is not None:
                image_data_1.append(np.array(img)/255)
                
            
image_data_1=np.array(image_data_1)
#label_1=np.array(label_1)
#final_output=[[1,0,0,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0,0,0],[0,0,1,0,0,0,0,0,0,0],[0,0,0,1,0,0,0,0,0,0],[0,0,0,0,1,0,0,0,0,0],[0,0,0,0,0,1,0,0,0,0],[0,0,0,0,0,0,1,0,0,0],[0,0,0,0,0,0,0,1,0,0],[0,0,0,0,0,0,0,0,1,0],[0,0,0,0,0,0,0,0,0,1]]
#one_hot_encoded_vector=[]
#for i in label:
    
 #   index=int(i)
  #  one_hot_encoded_vector.append(final_output[index])
#one_hot_encoded_vector=np.array(one_hot_encoded_vector) 


CREATING THE CLASS TO MAKE THE LAYERS OF THE NEURAL NETWORK.

In [4]:
import numpy as np

class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.weights_hidden1 = np.random.randn(self.hidden_size[0], self.input_size)
        self.bias_hidden1 = np.zeros((self.hidden_size[0], 1))-0.3
        
        self.weights_hidden2 = np.random.randn(self.hidden_size[1], self.hidden_size[0])
        self.bias_hidden2 = np.zeros((self.hidden_size[1], 1))-0.3
        
        self.weights_hidden3 = np.random.randn(self.hidden_size[2], self.hidden_size[1])
        self.bias_hidden3 = np.zeros((self.hidden_size[2], 1))-0.3
        
        self.weights_output = np.random.randn(self.output_size, self.hidden_size[2])
        self.bias_output = np.zeros((self.output_size, 1))-0.3
        
    def relu(self, x):
        return np.maximum(0, x)
    
    def softmax(self, x):
        x -= np.max(x, axis=0)
        exp_vals = np.exp(x)
        return exp_vals / np.sum(exp_vals, axis=0)
    
    def flatten(self, x):
        Flattend_layer_output=[]
        for data in x:
            Flattend_layer_output.append(np.ravel(data))
        return (np.array(Flattend_layer_output).T)
    
    def forward_propagation(self, x):
        self.input = self.flatten(x)
        
        self.hidden1_input = np.dot(self.weights_hidden1, self.input) + self.bias_hidden1
        self.hidden1_output = np.tanh(self.hidden1_input)
        
        self.hidden2_input = np.dot(self.weights_hidden2, self.hidden1_output) + self.bias_hidden2
        self.hidden2_output = np.tanh(self.hidden2_input)
        
        self.combined_output=self.hidden2_output+self.hidden1_output
        
        self.hidden3_input = np.dot(self.weights_hidden3, self.combined_output) + self.bias_hidden3
        self.hidden3_output = np.tanh(self.hidden3_input)
        
        self.output_input = np.dot(self.weights_output, self.hidden3_output) + self.bias_output
        
        self.output_output = self.softmax(self.output_input)
        
        
    def mse_loss(self, predicted, target):
        return np.mean((predicted - target)**2)
    
    def backpropagation(self, x, y, learning_rate):
        m = x.shape[1]  # number of training examples
        
        d_loss_predicted = 2 * (self.output_output - y) / 2500
        
        dw_output = np.dot(d_loss_predicted, self.hidden3_output.T)
        db_output = np.sum(d_loss_predicted, axis=1, keepdims=True)
        
        d_loss_hidden3 = np.dot(self.weights_output.T, d_loss_predicted)
        d_loss_hidden3=d_loss_hidden3*(1-(np.tanh(self.hidden3_input))**2) 
        
        dw_hidden3 = np.dot(d_loss_hidden3, self.hidden2_output.T)
        db_hidden3 = np.sum(d_loss_hidden3, axis=1, keepdims=True)
        
        d_loss_hidden2 = np.dot(self.weights_hidden3.T, d_loss_hidden3)
        d_loss_hidden2=d_loss_hidden2*(1-(np.tanh(self.hidden2_input)))
        
        dw_hidden2 = np.dot(d_loss_hidden2, self.hidden1_output.T)
        db_hidden2 = np.sum(d_loss_hidden2, axis=1, keepdims=True)
        
        d_loss_hidden1 = np.dot(self.weights_hidden2.T, d_loss_hidden2)
        d_loss_hidden1=d_loss_hidden1*(1-np.tanh(self.hidden1_input)**2)
        d_loss_hidden1_to_3=np.dot(self.weights_hidden3.T,d_loss_hidden3)
        d_loss_hidden1_to_3=d_loss_hidden1_to_3*(1-np.tanh(self.hidden1_input)**2)
        
        dw_hidden1 = np.dot(d_loss_hidden1, self.flatten(x).T)+np.dot(d_loss_hidden1_to_3,self.flatten(x).T)
        db_hidden1 = np.sum(d_loss_hidden1+d_loss_hidden1_to_3, axis=1, keepdims=True)
        
        self.weights_output -= learning_rate * dw_output
        self.bias_output -= learning_rate * db_output
        
        self.weights_hidden3 -= learning_rate * dw_hidden3
        self.bias_hidden3 -= learning_rate * db_hidden3
        
        self.weights_hidden2 -= learning_rate * dw_hidden2
        self.bias_hidden2 -= learning_rate * db_hidden2
        
        self.weights_hidden1 -= learning_rate * dw_hidden1
        self.bias_hidden1 -= learning_rate * db_hidden1
    
    
            
    def predict(self, x):
        self.forward_propagation(x)
        return self.output_output
    def calculate_accuracy(self,predictions, targets):
        predicted_labels = np.argmax(predictions, axis=1)
        true_labels = np.argmax(targets, axis=1)
        
    
        correct_predictions = np.sum(np.equal(predicted_labels,true_labels))
        total_predictions = len(targets)
    
        accuracy = correct_predictions / 2500
        return accuracy
    def train(self, x, y, epochs, learning_rate):
        for epoch in range(epochs):
            
            self.forward_propagation(x)
            self.backpropagation(x, y, learning_rate)
            loss = self.mse_loss(self.output_output, y)
            
            
            accuracy=self.calculate_accuracy(self.output_output.T,one_hot_encoded_vector)
            print(f"Epoch {epoch+1}/{epochs}, Loss: {loss:.4f},ACCURACY :{accuracy*100} ")
    def test(self,x):
        self.forward_propagation(x)
        
        predictions=np.argmax(self.output_output,axis=1)
        print(predictions)
        return(predictions)


input_size = 784
hidden_size = [8,8,8]
output_size = 10

x = image_data
y = one_hot_encoded_vector.T

nn = NeuralNetwork(input_size, hidden_size, output_size)

# Train the neural network
nn.train(x, y, epochs=100, learning_rate=0.3)

# TEST THE DATASET
nn.test(image_data_1)


Epoch 1/100, Loss: 0.1299,ACCURACY :11.559999999999999 
Epoch 2/100, Loss: 0.1200,ACCURACY :11.08 
Epoch 3/100, Loss: 0.1121,ACCURACY :9.8 
Epoch 4/100, Loss: 0.1085,ACCURACY :11.799999999999999 
Epoch 5/100, Loss: 0.1052,ACCURACY :12.24 
Epoch 6/100, Loss: 0.1034,ACCURACY :13.600000000000001 
Epoch 7/100, Loss: 0.1008,ACCURACY :14.56 
Epoch 8/100, Loss: 0.1000,ACCURACY :16.08 
Epoch 9/100, Loss: 0.0969,ACCURACY :17.44 
Epoch 10/100, Loss: 0.0962,ACCURACY :20.48 
Epoch 11/100, Loss: 0.0943,ACCURACY :16.88 
Epoch 12/100, Loss: 0.0943,ACCURACY :21.2 
Epoch 13/100, Loss: 0.0946,ACCURACY :13.639999999999999 
Epoch 14/100, Loss: 0.0961,ACCURACY :17.24 
Epoch 15/100, Loss: 0.0921,ACCURACY :19.24 
Epoch 16/100, Loss: 0.0897,ACCURACY :20.72 
Epoch 17/100, Loss: 0.0892,ACCURACY :21.48 
Epoch 18/100, Loss: 0.0894,ACCURACY :21.92 
Epoch 19/100, Loss: 0.0874,ACCURACY :23.44 
Epoch 20/100, Loss: 0.0865,ACCURACY :24.88 
Epoch 21/100, Loss: 0.0874,ACCURACY :23.04 
Epoch 22/100, Loss: 0.0884,ACCURACY 

array([ 79, 372,  62, 595, 318, 430, 215, 282,   1, 279])